<a href="https://colab.research.google.com/github/Heisneky/CSTU_chatbot_RAG/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ใช้ TiDB
cloud ที่รองรับ vector

## installation library

In [ ]:
pip install mysql-connector-python sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

## เชื่อมต่อฐานข้อมูล

ในส่วนนี้ผมใช้ cloud database(TiDB) ปรับเปลี่ยนตามที่อยากจะเก็บเลยครับอาจารย์

In [ ]:
import mysql.connector
# connect to TiDB
# ssl_CA นำไฟล์ isrgrootx1.pem มาติดตั้ง เนื่องจากผมใช้งาน cloud database
conn = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "3B1P9hGu3KA2VcL.root",
  password = "JAvNOce19Qef0hCM",
  database = "vector_db",
  ssl_ca = "/content/isrgrootx1.pem",
  ssl_verify_cert = True,
  ssl_verify_identity = True
)

## extract text from PDF

### ติดตั้ง library

In [ ]:
pip install pdfplumber ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.7 MB/s eta 0:00:00


### ไฟล์ CSTUguide สำหรับข้อมูลแนะนำทั่วไปนักศึกษา

In [ ]:
import pdfplumber
import re
from ftfy import fix_text

# แก้คำ
def clean_text(text):
    text = fix_text(text)
    text = text.replace('\uf70a', '่')
    text = text.replace('\uf70b', '้')
    text = text.replace('\uf70c', '๊')
    text = text.replace('\uf70d', '๋')
    text = text.replace('\uf70e', '์')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

documents = []
# ตำแหน่งไฟล์แก้ตรงนี้
with pdfplumber.open("CSTUguide.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines = text.split('\n')

            paragraph_blocks = []
            current_block = ""

            for line in lines:
                line = clean_text(line)
                if len(line.strip()) == 0:
                    continue

                # ถ้าจบบรรทัดตามธรรมชาติ หรือยาวพอ
                if line.endswith(('.', '!', '?', '”')) or len(line) > 100:
                    current_block += " " + line
                    paragraph_blocks.append(current_block.strip())
                    current_block = ""
                else:
                    current_block += " " + line

            # บรรทัดสุดท้าย
            if current_block.strip():
                paragraph_blocks.append(current_block.strip())

            # เพิ่ม [XXXX] ลงไปในแต่ละข้อความ ปรับเปลี่ยนตามความเหมาะสม
            for p in paragraph_blocks:
                if len(p) > 30:
                    documents.append(f"[คู่มือนักศึกษา] {p}")

# ตัดข้อความซ้ำ หากมีซ้ำ
documents = list(set(documents))

print(f"📄 จำนวนกลุ่มข้อความทั้งหมด: {len(documents)}")
print("\nตัวอย่าง 5 กลุ่มข้อความแรก:\n")
for i, line in enumerate(documents[:5], 1):
    print(f"{i}. {line}")


📄 จำนวนกลุ่มข้อความทั้งหมด: 53

ตัวอย่าง 5 กลุ่มข้อความแรก:

1. [คู่มือนักศึกษา] 17 4.2) วิชาโทการออกแบบและพัฒนาผลิตภัณฑ์ดิจิทัล (Digital Product Design and Development - DPDD) นักศึกษาผู้ประสงค์จะศึกษาวิชาโทการออกแบบและพัฒนาผลิตภัณฑ์ดิจิทัลเป็น วิชาโท ต้องศึกษารายวิชาด้านการออกแบบและพัฒนาผลิตภัณฑ์ดิจิทัลไม่น้อยกว่า 15 หน่วยกิต ตามเงื่อนไขดังต่อไปนี้ รหัสวิชา ชื่อวิชา หน่วยกิต (บรรยาย-ปฏบิ ัต-ิ ศึกษาด้วยตัวเอง) คพ.287 การออกแบบสุนทรียภาพทางทัศนะ 3 (3-0-6) CS 287 Visual Design คพ.342 การวิเคราะห์และเทคโนโลยีการตลาดดิจิทัล 3 (3-0-6) CS 342 Digital Marketing Technology and Analytics คพ.355 กลยุทธ์และการเปลี่ยนแปลงสู่ดิจิทัล 3 (3-0-6) CS 355 Digital Strategy and Transformation คพ.384 การปฏิสัมพันธ์ระหว่างมนุษย์และคอมพิวเตอร์ 3 (3-0-6) CS 384 Human - Computer Interaction คพ.385 โครงงานทางการออกแบบผลิตภัณฑ์และบริการด้านดิจิทัล 3 (3-0-6) CS 385 Digital Product and Service Design Project ทั้งนี้ กรณีนักศึกษาได้ศึกษารายวิชาเหล่านี้ในหมวดวิชาเฉพาะแล้ว จะไม่สามารถนํา รายวิชาดังกล่าวมานับหน่วยกิตใ

### ไฟล์ comsci2561 สำหรับหลักสูตรปี 2561

In [ ]:
import pdfplumber
import re
from ftfy import fix_text

def clean_text(text):
    text = fix_text(text)
    text = text.replace('\uf70a', '่')
    text = text.replace('\uf70b', '้')
    text = text.replace('\uf70c', '๊')
    text = text.replace('\uf70d', '๋')
    text = text.replace('\uf70e', '์')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

documents = []
YEAR = "2561"

with pdfplumber.open("comsci2561.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines = text.split('\n')

            paragraph_blocks = []
            current_block = ""

            for line in lines:
                line = clean_text(line)
                if len(line.strip()) == 0:
                    continue

                # ถ้าจบบรรทัดตามธรรมชาติ หรือยาวพอ
                if line.endswith(('.', '!', '?', '”')) or len(line) > 100:
                    current_block += " " + line
                    paragraph_blocks.append(current_block.strip())
                    current_block = ""
                else:
                    current_block += " " + line

            # บรรทัดสุดท้าย
            if current_block.strip():
                paragraph_blocks.append(current_block.strip())

            # เพิ่ม [ปี XXXX] ลงไปในแต่ละข้อความ
            for p in paragraph_blocks:
                if len(p) > 30:
                    documents.append(f"[หลักสูตรปี {YEAR}] {p}")

# ตัดซ้ำ
documents = list(set(documents))

print(f"📄 จำนวนกลุ่มข้อความทั้งหมด: {len(documents)}")
print("\nตัวอย่าง 5 กลุ่มข้อความแรก:\n")
for i, line in enumerate(documents[:5], 1):
    print(f"{i}. {line}")


📄 จำนวนกลุ่มข้อความทั้งหมด: 776

ตัวอย่าง 5 กลุ่มข้อความแรก:

1. [หลักสูตรปี 2561] เลขประจำตัว ตำแหน่ง คุณวุฒิการศึกษา/สถาบัน/ ลำดับ ชื่อ - สกุล ประชาชน ทางวิชาการ ปีการศึกษาที่จบ 3 3830300353xxx ผู้ช่วยศาสตราจารย์ ดร.อรจริ า สิทธิศักด ิ์ - Ph.D. (Computer Sciences) /University of
2. [หลักสูตรปี 2561] qualitative forecasting and their applications in businesses.
3. [หลักสูตรปี 2561] ภาคเรียนที่ 1 หน่วยกิต คพ.265 ปัญญาประดิษฐ์เบื้องต้น (บังคับร่วม) 3 คพ.301 ข้อเสนอและการนำเสนอโครงงานทางวิทยาการคอมพิวเตอร์ (บังคับร่วม) 3 คพ.312 หลักการเรียนรู้เพื่อเทคโนโลยีการเรียนรู้ (บังคับเอก) 3 คพ.356 ระบบสารสนเทศเพื่อการบริหาร (บังคับเอก) 3 คพ.384 การปฏิสัมพันธ์ระหว่างมนุษย์และคอมพิวเตอร์ (บังคับร่วม) 3 คพ.385 วิทยาการข้อมูลที่มีมนุษย์เป็นศูนย์กลาง (บังคับเอก) 3 นักศึกษาอาจเลือกลงทะเบียนวิชาต่อไปน ี้ คพ.300 สัมมนาการโปรแกรมภาคปฏิบัต ิ (บังคับร่วม) 1 รวม 18 หรือ 19 สำหรับนักศึกษาที่จะศึกษาวชิ าโท จะต้องลงทะเบียนวิชาต่อไปน ี้ วิศวกรรมและการจัดการซอฟต์แวร์ วิทยาการข้อมูล หน่วยกิต คพ.266 กระบวนการซอฟต์แ

### ไฟล์ comsci2566 สำหรับหลักสูตรปี 2566

In [ ]:
import pdfplumber
import re
from ftfy import fix_text

def clean_text(text):
    text = fix_text(text)
    text = text.replace('\uf70a', '่')
    text = text.replace('\uf70b', '้')
    text = text.replace('\uf70c', '๊')
    text = text.replace('\uf70d', '๋')
    text = text.replace('\uf70e', '์')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

documents = []
YEAR = "2566"

with pdfplumber.open("comsci2566.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines = text.split('\n')

            paragraph_blocks = []
            current_block = ""

            for line in lines:
                line = clean_text(line)
                if len(line.strip()) == 0:
                    continue

                if line.endswith(('.', '!', '?', '”')) or len(line) > 100:
                    current_block += " " + line
                    paragraph_blocks.append(current_block.strip())
                    current_block = ""
                else:
                    current_block += " " + line

            if current_block.strip():
                paragraph_blocks.append(current_block.strip())

            # เพิ่ม [ปี XXXX] ลงไปในแต่ละข้อความ
            for p in paragraph_blocks:
                if len(p) > 30:
                    documents.append(f"[หลักสูตรปี {YEAR}] {p}")

# ตัดซ้ำ
documents = list(set(documents))

print(f"📄 จำนวนกลุ่มข้อความทั้งหมด: {len(documents)}")
print("\nตัวอย่าง 5 กลุ่มข้อความแรก:\n")
for i, line in enumerate(documents[:5], 1):
    print(f"{i}. {line}")


📄 จำนวนกลุ่มข้อความทั้งหมด: 274

ตัวอย่าง 5 กลุ่มข้อความแรก:

1. [หลักสูตรปี 2566] อย่างเปน ระบบ มีการเตรียมความพร้อมให้บัณฑิตมีความรู้รอบเกี่ยวกับการเปลี่ยนแปลงของเศรษฐกิจและ สังคมโลก สามารถสื่อสารและปรับตัวเท่าทันต่อการเปลี่ยนแปลงของเศรษฐกิจและสังคมโลกได้อย่าง เหมาะสมและยั่งยืน 3.3 วัตถุประสงค์ของหลักสูตร เพื่อให้บัณฑิตที่สำเร็จการศึกษาในหลักสูตรมีคุณลักษณะ ดังนี้ 1) มีความรู้ความสามารถทางวิชาการ สามารถคิดแก้ปญหาอย่างเปนระบบ 2) มีความรู้ความสามารถทางวิชาการ สามารถนำไปต่อยอดเปนทฤษฎีหรือแนวคิดใหม่ ใน การศึกษาต่อในระดับสูงได้ 3) มีความคิดริเริ่มสร้างสรรค์ในการนำความรู้ด้านวิทยาการคอมพิวเตอร์ไปบูรณาการกับศาสตร์ แขนงอื่นๆ พัฒนาเปนนวัตกรรมทางด้านสังคมและด้านธุรกิจที่ตอบสนองความต้องการของ ผู้ใช้ สามารถแก้ปญหาได้อย่างเหมาะสมและยั่งยืน 4) มีความเปนบัณฑิตนักปฏิบัติ สามารถปฏิบัติงานได้จริง มีทักษะและวิสัยทัศน์ทางอาชีพใน อุตสาหกรรมซอฟต์แวร์และดิจิทัล 5) มีคุณธรรมและจริยธรรมซึ่งสอดคล้องกับปณิธานของมหาวิทยาลัย 6) มีความพร้อมในการแข่งขันและรู้รอบเกี่ยวกับการเปลี่ยนแปลงของเศรษฐกิจและสังคมโลก 

## embed

In [ ]:
from sentence_transformers import SentenceTransformer

# โมเดลที่ใช้แปลง word to vec
embedder = SentenceTransformer("BAAI/bge-m3")
# text = "I love Machine Learning!"
# embedding = embedder.encode(text).tolist()
# embedding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# ไว้ดูขนาด
len(documents)

53

In [ ]:
documents = [
   "ใส่ประโยคที่ยังไม่แปลง",
   "ใส่ประโยคที่ยังไม่แปลง"
]

# กรณีที่เพิ่มข้อมูลเอง

### ลบตาราง

In [ ]:
cursor = conn.cursor()

try:
    cursor.execute("DROP TABLE IF EXISTS documents;")
    conn.commit()
    print("✅ ตาราง 'documents' ถูกลบแล้ว")

except Exception as e:
    print("❌ ไม่สามารถลบตารางได้:", e)

cursor.close()


✅ ตาราง 'documents' ถูกลบแล้ว


True

## เพิ่มลงตาราง

In [ ]:
import json

In [ ]:

# แปลง documents เป็น vector ก่อนส่งไปเก็บบน TiDB
def add_document(conn, text):
    cur = conn.cursor()
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (document, embedding) VALUES (%s, %s)", (text, json.dumps(embedding)))
    conn.commit()
    cur.close()

for doc in documents:
    add_document(conn, doc)

## ทดสอบการใช้งาน

In [ ]:
# import json

# คัดคำตอบมาตรียม ให้ llm
def query_tidb(conn, query_text, k=5):
    query_embedding = json.dumps(embedder.encode(query_text).tolist())
    # print(query_embedding)

    cur = conn.cursor()

# k คือจำนวนคำตอบที่ดึงมา
    sql_query = f"""
        SELECT document, vec_cosine_distance(embedding, '{query_embedding}') AS distance
        FROM documents
        ORDER BY distance
        LIMIT {k}
    """

    cur.execute(sql_query)
    results = cur.fetchall()
    cur.close()

    return [(content, distance) for content, distance in results]
# คำถาม
results = query_tidb(conn, "วิทย์คอมต้องเรียนกี่หน่วยกิต อะไรบ้าง")
results

[('[หลักสูตรปี 2561] การศึกษาปกติ และอย่างมากไม่เกิน 16 ภาคการศึกษาปกต ิ 3.1.2 โครงสร้างหลักสูตร นักศึกษาจะต้องจดทะเบียนศึกษารายวิชา รวมไม่น้อยกว่า 129 หน่วยกิต โดยศึกษารายวิชาต่างๆ ครบตามโครงสร้าง องค์ประกอบและข้อกำหนดของหลักสูตรดังนี้ 1. วิชาศึกษาทั่วไป 30 หน่วยกิต 2. วิชาเฉพาะ ไม่น้อยกว่า 93 หน่วยกิต 2.1 วิชาพื้นฐานทางด้านคณิตศาสตร์ 9 หน่วยกิต 2.2 วิชาแกนบังคับร่วมในสาขา 36 หน่วยกิต - กลุ่มประเด็นด้านองค์การและระบบสารสนเทศ 6 หน่วยกิต - กลุ่มเทคโนโลยีเพื่องานประยุกต์ 14 หน่วยกิต - กลุ่มเทคโนโลยีและวิธีการทางซอฟต์แวร์ 10 หน่วยกิต - กลุ่มโครงสร้างพื้นฐานของระบบ 3 หน่วยกิต - กลุ่มวิชาแกนคณิตศาสตร์ดสิ ครตี 3 หน่วยกิต 2.3 วิชาบังคับเอก 2.3.1 วิชาเอก คอมพิวเตอร์และวิทยาการสารสนเทศ 30 หน่วยกิต - กลุ่มเทคโนโลยีเพื่องานประยุกต์ 3 หน่วยกิต - กลุ่มเทคโนโลยีและวิธีการทางซอฟต์แวร์ 3 หน่วยกิต 15',
  0.31107093956851417),
 ('[หลักสูตรปี 2566] 4.3 โครงสร้างหลักสูตร รายวิชา และหน่วยกิต 4.3.1 หลักสูตร 4.3.1.1 จำนวนหน่วยกิตรวม จำนวนหน่วยกิตรวมตลอดหลักสูตร 123 หน่วยกิต 4.3.1.2 โครงสร้างหลักสูตร นักศึกษา

## ใช้งานโปรแกรม

หากจะนำไปใช้ให้แก้ไข API key

In [ ]:
import requests

# typhoon-v2.1-12b-instruct
# typhoon-v2-8b-instruct
# typhoon-v2-70b-instruct

TYHOON_API_URL = "https://api.opentyphoon.ai/v1/chat/completions"
API_KEY = "sk-9vbfWrMPYBu1BXkCRGImQb53YoUbvw9iECjfZUNuwTg3ByWn"  # ใส่ API Key
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

def generate_response(conn, query_text):
    retrived_docs = query_tidb(conn, query_text)
    context = "\n".join([doc[0] for doc in retrived_docs])

    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query_text}"

    payload = {
        "model": "typhoon-v2-70b-instruct",
        "messages": [
            {"role": "system", "content": "คุณคือเจ้าหน้าที่มหาวิทยาลัยธรรมศาสตร์รังสิต ที่มีหน้าที่ตอบคำถามอย่างสุภาพ หากข้อมูลไม่เพียงพอหรือคำถามไม่ชัดเจนให้บอกผู้ถามให้ถามอีกครั้ง และหากในคำถามเรื่องหลักสูตรการเรียนไม่ได้ระบุปีหลักสูตรให้เข้าใจเลยว่าถามถึงหลักสูตรปี2566"},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 500
    }

    response = requests.post(TYHOON_API_URL, json=payload, headers=HEADERS)

    if response.status_code == 200:
        return response.json().get("choices", [{}])[0].get("message", {}).get("content", "No response from Typhoon API")
    else:
        return f"Error: {response.status_code}, {response.text}"

answer = generate_response(conn, "มีห้องสำหรับอะไรบ้างเพื่ออำนวจความสะดวกนักศึกษาวิทย์คอม")
print(answer)


สำหรับนักศึกษาวิทย์คอม มหาวิทยาลัยธรรมศาสตร์รังสิตมีห้องเรียนและห้องปฏิบัติการเพื่ออำนวยความสะดวกในการเรียนการสอนดังนี้:

1. ห้องเรียน: มีจำนวน 9 ห้อง ซึ่งมีอุปกรณ์เพื่ออำนวยความสะดวกในการสอน ได้แก่ เครื่องคอมพิวเตอร์เชื่อมต่อเข้าสู่ระบบเครือข่ายอินเทอร์เน็ตโดยเครื่องข่ายไร้สาย, เครื่อง LCD, เครื่องฉายสไลด์ Visual, และเครื่องเสียง

2. ห้องปฏิบัติการคอมพิวเตอร์: มีจำนวน 3 ห้อง เพื่อให้บริการการใช้งานเครื่องคอมพิวเตอร์สำหรับนักศึกษา

3. ห้องปฏิบัติการเฉพาะทาง: มีจำนวน 5 ห้อง ประกอบด้วย
   - ห้อง Internet of Things, Cloud, Networking and Knowledge Integration Technology Laboratory (TONKIT Lab)
   - ห้อง Data Innovation and Artificial Intelligence Lab (DI-Al Lab)
   - ห้อง Virtualization Architecture and ScalABle Infrastructure Lab (Vasabi Lab)
   - ห้อง วิจัยด้านวิศวกรรมซอฟต์แวร์ (Software Engineering Lab)
   - ห้อง Multimedia Analysis & Computer Human Interaction Lab (MACHI Lab)

ห้องปฏิบัติการเฉพาะทางเหล่านี้เปิดให้บริการตลอด 24 ชั่วโมง สำหรับนักศึกษาที่ได้แจ้งความประสงค์เข้าใช้และได้รั

## spam ถาม

In [ ]:
answer = generate_response(conn, "นักศึกษาวิทย์คอมจะมีสิทธิพิเศษอะไรบ้าง")
print(answer)

นักศึกษาวิทยาการคอมพิวเตอร์ที่มหาวิทยาลัยธรรมศาสตร์ จะมีสิทธิพิเศษดังนี้:

1. พิมพ์งานฟรีสูงสุด 300 บาทต่อเทอม แบ่งเป็นสิทธิพิมพ์งานฟรี 200 บาท และถ้าประเมินการเรียนการสอนในช่วงปลายภาคการศึกษาจะได้สิทธิเพิ่มอีก 100 บาท
2. ใช้งาน Office 365 เช่น Microsoft Office Online, OneDrive และอื่นๆ อีกมากมาย รวมไปถึงสามารถโหลดโปรแกรม Adobe Photoshop ได้ฟรี โดยใช้ E-mail นักศึกษา
3. ยืมอุปกรณ์ต่างๆ สำหรับใช้ภายในบริเวณหอสมุดป๋วย อึ๊งภากรณ์ เช่น ไอแพด ปลั๊กพ่วงสายไฟ ที่ตั้งอ่านหนังสือ หูฟัง สายชาร์จ Lightning เป็นต้น
4. ใช้งาน Canva Pro ได้ฟรีโดยใช้ E-mail นักศึกษา
5. ใช้สิทธิรักษากรณีเจ็บป่วยทั่วไปได้ที่โรงพยาบาลธรรมศาสตร์ โดยสามารถใช้สิทธิได้ 5,000 บาท/ครั้ง และรวมไม่เกิน 20,000 บาท/ปีการศึกษา และสิทธิทันตกรรมขั้นพื้นฐานฟรี ได้แก่ ตรวจฟัน อุดฟัน ถอนฟัน ขูดหินปูน และ X-ray ฟิล์มเล็ก ไม่เกิน 300 บาท/ครั้ง/คน และไม่เกิน 3 ครั้ง/ปีการศึกษา
6. เข้าใช้งานฟิตเนสและสนามกีฬาของมหาวิทยาลัยธรรมศาสตร์ได้ฟรี ได้แก่ ฟิตเนสหอพักโซน B และ C และ TU Fitness รวมไปถึงสามารถเข้าใช้สระว่ายน้ำศูนย์กีฬาทางน้ำและ X-treme 

In [ ]:
answer = generate_response(conn, "ต้องเรียนอะไรบ้างถึงจะจบ")
print(answer)

ตามหลักสูตรปี 2566 นักศึกษาจะต้องเรียนรายวิชาต่างๆ ครบตามโครงสร้างหลักสูตร ซึ่งประกอบด้วย:

1. วิชาศึกษาทั่วไป 30 หน่วยกิต
2. วิชาเฉพาะ ไม่น้อยกว่า 93 หน่วยกิต
   - วิชาพื้นฐานทางด้านคณิตศาสตร์ 9 หน่วยกิต
   - วิชาแกนบังคับร่วมในสาขา 36 หน่วยกิต
     - กลุ่มประเด็นด้านองค์การและระบบสารสนเทศ 6 หน่วยกิต
     - กลุ่มเทคโนโลยีเพื่องานประยุกต์ 14 หน่วยกิต
     - กลุ่มเทคโนโลยีและวิธีการทางซอฟต์แวร์ 10 หน่วยกิต
     - กลุ่มโครงสร้างพื้นฐานของระบบ 3 หน่วยกิต
     - กลุ่มวิชาแกนคณิตศาสตร์ดสิ ครตี 3 หน่วยกิต
   - วิชาบังคับเอก 30 หน่วยกิต
     - วิชาเอก คอมพิวเตอร์และวิทยาการสารสนเทศ 30 หน่วยกิต
       - กลุ่มเทคโนโลยีเพื่องานประยุกต์ 3 หน่วยกิต
       - กลุ่มเทคโนโลยีและวิธีการทางซอฟต์แวร์ 3 หน่วยกิต

นอกจากนี้ นักศึกษายังต้องสอบผ่านและได้รับหน่วยกิตสะสมรายวิชาครบตามโครงสร้างหลักสูตร และมีหน่วยกิตสะสมไม่ต่ำกว่า 123 หน่วยกิต รวมถึงต้องปฏิบัติตามเงื่อนไขอื่นๆ ที่คณะวิทยาศาสตร์และเทคโนโลยี และมหาวิทยาลัยธรรมศาสตร์กำหนด


In [ ]:
answer = generate_response(conn, "วิชาที่กำหนดให้ผ่าน C มีอะไรบ้าง")
print(answer)

ตามหลักสูตรปี 2566 นักศึกษาจะต้องสอบไล่ได้ไม่ต่ำกว่าระดับ C ในรายวิชา คพ.101, คพ.102 และ คพ.111 ครับ


In [ ]:
answer = generate_response(conn, "หากจะลงวิชาสหกิจต้องลงตัวไหนก่อน")
print(answer)

หากต้องการลงวิชาสหกิจ ต้องลงวิชา คพ.304 สหกิจศึกษา 1 ก่อน แล้วจึงสามารถลงวิชา คพ.404 สหกิจศึกษา 2 ได้ โดยต้องสอบผ่าน คพ.304 ด้วยเกรดไม่ต่ำกว่า C


In [ ]:
answer = generate_response(conn, "ในเทอมแรกควรลงวิชาอะไรดี")
print(answer)

ตามข้อมูลที่ให้มาในหลักสูตรปี 2561 สำหรับเทอมแรก นักศึกษาควรลงวิชาดังต่อไปนี้:

1. คพ.101 โครงสร้างแบบไม่ต่อเนื่อง (บังคับร่วม) 3 หน่วยกิต
2. คพ.102 พื้นฐานการแก้ปัญหาและการโปรแกรมคอมพิวเตอร์ (บังคับร่วม) 3 หน่วยกิต
3. มธ.100 พลเมืองกับการลงมือแก้ปัญหา 3 หน่วยกิต
4. มธ.102 ทักษะชีวิตทางสังคม 3 หน่วยกิต
5. มธ.104 การคิด อ่าน และเขียนอย่างมีวิจารณญาณ 3 หน่วยกิต
6. มธ.050 การพัฒนาทักษะภาษาอังกฤษ (หรือ) 3 หน่วยกิต
7. มธ.105 ทักษะการสื่อสารด้วยภาษาอังกฤษ 3 หน่วยกิต
8. จ.252 จิตวิทยาความสัมพันธ์ระหว่างบุคคล 3 หน่วยกิต

รวมทั้งหมด 21 หน่วยกิต

หากมีข้อสงสัยเพิ่มเติมหรือคำถามที่ไม่ชัดเจน กรุณาถามอีกครั้งค่ะ


In [ ]:
answer = generate_response(conn, "ปีการศึกษาที่ 1เทอม 1 ต้องลงวิชาไหนบ้างสำหรับนักศึกษาหลักสูตร 2566")
print(answer)

สำหรับนักศึกษาหลักสูตรปี 2566 ในปีการศึกษาที่ 1 เทอม 1 ต้องลงวิชาดังต่อไปนี้:

1. คพ.100 การพัฒนาเว็บแอปพลิเคชันเบื้องต้น (วิชาเฉพาะด้าน, บังคับร่วม) 3 หน่วยกิต
2. คพ.101 โครงสร้างแบบไม่ต่อเนื่อง (วิชาแกน) 3 หน่วยกิต
3. คพ.102 พื้นฐานการแก้ปัญหาและการโปรแกรม (วิชาเฉพาะด้าน, บังคับร่วม) 3 หน่วยกิต
4. มธ.100 พลเมืองกับการลงมือแก้ปัญหา (ศึกษาทั่วไป) 3 หน่วยกิต
5. มธ.103 ชีวิตกับความยั่งยืน (ศึกษาทั่วไป) 3 หน่วยกิต
6. สษ.105 ทักษะการสื่อสารทางภาษาอังกฤษ (ศึกษาทั่วไป) 3 หน่วยกิต
7. ส.216 สถิติสำหรับสังคมศาสตร์ 1 (วิชาแกน) 3 หน่วยกิต

รวมทั้งหมด 21 หน่วยกิต


In [ ]:
answer = generate_response(conn, "ค่าเทอมแพงไหม")
print(answer)

ขออภัยครับ/ค่ะ คำถามของคุณไม่ชัดเจนเกี่ยวกับค่าเทอมที่คุณต้องการทราบ คุณหมายถึงค่าเทอมของหลักสูตรใดในปีไหน หรือมีข้อมูลเพิ่มเติมเกี่ยวกับคำถามของคุณหรือไม่ครับ/ค่ะ?


In [ ]:
answer = generate_response(conn, "ค่าเทอมของวิทย์คอมแพงไหม")
print(answer)

จากข้อมูลที่ให้มา ค่าใช้จ่ายตลอดหลักสูตรสำหรับโครงการปกติ (วิชาเอกคอมพิวเตอร์และวิทยาการสารสนเทศ) สำหรับนักศึกษาไทยอยู่ที่ประมาณ 138,400 บาท ซึ่งแบ่งเป็นระยะเวลา 4 ปี ทำให้ค่าใช้จ่ายต่อปีอยู่ที่ประมาณ 34,600 บาท สำหรับโครงการพิเศษ (วิชาเอกคอมพิวเตอร์ประยุกต์) ค่าใช้จ่ายตลอดหลักสูตรสำหรับนักศึกษาไทยอยู่ที่ประมาณ 302,320 บาท หรือประมาณ 75,580 บาทต่อปี

ดังนั้น ค่าเทอมของหลักสูตรวิทยาการคอมพิวเตอร์ขึ้นอยู่กับประเภทโครงการที่เลือก หากเลือกโครงการปกติ ค่าใช้จ่ายจะอยู่ที่ประมาณ 34,600 บาทต่อปี แต่ถ้าเลือกโครงการพิเศษ ค่าใช้จ่ายจะอยู่ที่ประมาณ 75,580 บาทต่อปี


In [ ]:
answer = generate_response(conn, "วิชาไหนที่หลักสูตร 2561 และ2566 เรียนไม่เหมือนกัน")
print(answer)

จากข้อมูลที่ให้มา มีวิชาที่หลักสูตร 2561 และ 2566 เรียนไม่เหมือนกันดังนี้:

1. คพ.333 - ในหลักสูตร 2561 ชื่อวิชา "สถาปตยกรรมบริการ" แต่ในหลักสูตร 2566 ชื่อวิชา "สถาปตยกรรมซอฟต์แวร์"
2. คพ.352 - ในหลักสูตร 2561 ชื่อวิชา "กระบวนการตัดสินใจทางธุรกิจ" แต่ในหลักสูตร 2566 ชื่อวิชา "การวิเคราะห์และเทคโนโลยี"
3. คพ.369 - ในหลักสูตร 2561 ชื่อวิชา "การพัฒนาเว็บแอปพลิเคชัน" แต่ในหลักสูตร 2566 ชื่อวิชา "โครงงานทางวิศวกรรมซอฟต์แวร์"
4. คพ.377 - ในหลักสูตร 2561 ชื่อวิชา "การเรียนรู้ของเครื่องจักร" แต่ในหลักสูตร 2566 ชื่อวิชา "การเรียนรู้ของเครื่อง"
5. คพ.299 - ในหลักสูตร 2561 ชื่อวิชา "คอมพิวเตอร์กราฟก 1" แต่ในหลักสูตร 2566 ชื่อวิชา "คอมพิวเตอร์กราฟก 3"
6. คพ.301 - ในหลักสูตร 2561 ชื่อวิชา "ข้อเสนอและการนำเสนอ" แต่ในหลักสูตร 2566 ชื่อวิชา "การเตรียมความพร้อมสำหรับ"
7. คพ.345 - ในหลักสูตร 2561 ชื่อวิชา "การเรียนรู้ของเครื่องจักรและการ" แต่ในหลักสูตร 2566 ชื่อวิชา "การเรียนรู้ของเครื่องเชิงประยุกต์"

นอกจากนี้ยังมีวิชาที่เปิดเพิ่มในหลักสูตร 2566 เช่น คพ.100, คพ.140, คพ.180, คพ.223, คพ.224, คพ.264,

In [ ]:
answer = generate_response(conn, "วิชาTU มีตัวอะไรบ้าง")
print(answer)

จากข้อมูลที่ให้มา วิชา TU ที่ปรากฏในหลักสูตรปี 2561 มีดังนี้:

1. TU 100 - Civic Engagement (พลเมืองกับการลงมือแก้ปัญหา)
2. TU 101 - Thailand, ASEAN, and the World (โลก, อาเซียน และไทย)
3. TU 102 - Social Life Skills (ทักษะชีวิตทางสังคม)
4. TU 103 - Life and Sustainability (ชีวิตกับความยั่งยืน)
5. TU 104 - Critical Thinking, Reading, and Writing (การคิด อ่าน และเขียนอย่างมีวิจารณญาณ)
5. TU 105 - Communication Skills in English (ทักษะการสื่อสารด้วยภาษาอังกฤษ)
6. TU 106 - Creativity and Communication (ความคิดสร้างสรรค์และการสื่อสาร)
7. TU 107 - Digital Skill and Problem Solving (ทักษะดิจิทัลกับการแก้ปัญหา)
8. TU 108 - Self-Development and Management (การพัฒนาและจัดการตนเอง)
9. TU 109 - Innovation and Entrepreneurial Mindset (นวัตกรรมกับกระบวนการคิดของผู้ประกอบการ)
10. TU 122 - Law in Everyday Life (กฎหมายในชีวิตประจำวัน)
11. TU 201 - Financial Literacy for Individuals (ความรู้ทางการเงินสำหรับบุคคล)
12. TU 202 - Complete Investment (ครบเครื่องเรื่องลงทุน)
13. TU 301 - Investment in the St

In [ ]:
answer = generate_response(conn, "อาจารย์ปกป้องนามสกุลอะไร")
print(answer)

อาจารย์ปกป้อง นามสกุล "ส่องเมือง" ครับ


In [ ]:
answer = generate_response(conn, "อาจารย์ปกป้องเป็นอาจารย์สาขาอะไร")
print(answer)

อาจารย์ปกป้อง ส่องเมือง เป็นอาจารย์สาขาวิชาวิทยาการคอมพิวเตอร์ โดยเฉพาะวิชาเอกคอมพิวเตอร์ประยุกต์ ที่ศูนย์รังสิตของมหาวิทยาลัยธรรมศาสตร์


In [ ]:
answer = generate_response(conn, "อาจารย์ปกรณ์ เป็นอาจารย์สาขาอะไร")
print(answer)

จากข้อมูลที่ให้มา มีอาจารย์ชื่อปกรณ์ 2 ท่าน ได้แก่:

1. รองศาสตราจารย์ ปกรณ์ เสริมสุข ซึ่งสอนในสาขาวิทยาการคอมพิวเตอร์
2. ผู้ช่วยศาสตราจารย์ ดร.ปกรณ์ ลี้สุทธิพรชัย ซึ่งสอนในสาขาวิศวกรรมคอมพิวเตอร์

หากต้องการข้อมูลเพิ่มเติมหรือความชัดเจนเกี่ยวกับอาจารย์ปกรณ์ท่านใด กรุณาสอบถามอีกครั้งครับ
